## Import stuff

In [1]:
import numpy as np
import pandas as pd
    

## Load training data

In [2]:
csv = pd.read_csv("train.csv")
y = csv["y"].to_numpy()
x_inputs = csv[["x1", "x2", "x3", "x4", "x5"]].to_numpy()

# apply all the operations, or actually get array A:
# apply all the operations, or actually get array A:
A = np.array([csv["x1"], csv["x2"], csv["x3"], csv["x4"], csv["x5"],
              np.square(csv["x1"]), np.square(csv["x2"]), np.square(csv["x3"]), np.square(csv["x4"]), np.square(csv["x5"]),
             np.exp(csv["x1"]), np.exp(csv["x2"]), np.exp(csv["x3"]), np.exp(csv["x4"]), np.exp(csv["x5"]),
             np.cos(csv["x1"]), np.cos(csv["x2"]), np.cos(csv["x3"]), np.cos(csv["x4"]), np.cos(csv["x5"]),
             np.ones_like(csv["x1"])])


A_transpose = np.transpose(A)
x = np.linalg.lstsq(A_transpose, y)

C:\Users\andyt\AppData\Local\Temp/ipykernel_44452/3222968692.py:15: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x = np.linalg.lstsq(A_transpose, y)


   ## Export

In [3]:
with open("submit.csv", "w") as f:
    for i in x[0]:
        f.write(str(i) + "\n")